1. Create a Jupyter Notebook using Vertex AI Colab Enterprise, and write the code to do the following.
2. Import the following file into a BigQuery table: gs://labs.roitraining.com/data-to-ai-workshop/emergency_calls_response_times.csv

Create a data set

In [23]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS emergency_calls
OPTIONS(
location="us",
default_table_expiration_days=14 );

Query is running:   0%|          |

""


Import into a table

In [24]:
%%bigquery
CREATE OR REPLACE EXTERNAL TABLE `qwiklabs-gcp-01-a7e68aa7e5b0.emergency_calls.emergency_calls_table`
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/emergency_calls_response_times.csv'],
  skip_leading_rows = 1,
  allow_jagged_rows = false,
  allow_quoted_newlines = false,
  field_delimiter = ',',
  max_bad_records = 10
)


Query is running:   0%|          |

""


Perform feature engineering to transform table.

In [25]:
%%bigquery
CREATE OR REPLACE TABLE `qwiklabs-gcp-01-a7e68aa7e5b0.emergency_calls.emergency_calls_table_encoded` AS (
  SELECT
    -- Select all original columns, excluding those we are transforming to avoid duplication
    * EXCEPT (
      call_type, weather_condition, day_of_week, traffic_level
    ),

    -- Step 1: One-hot encode All String Fields
    -- Creates a new column for each category with a 1 or 0
    -- call_type
    CASE WHEN call_type = 'Police' THEN 1 ELSE 0 END AS call_type_police,
    CASE WHEN call_type = 'Medical' THEN 1 ELSE 0 END AS call_type_medical,
    CASE WHEN call_type = 'Fire' THEN 1 ELSE 0 END AS call_type_fire,
    CASE WHEN call_type = 'Rescue' THEN 1 ELSE 0 END AS call_type_rescue,
    -- weather_condition
    CASE WHEN weather_condition = 'Rainy' THEN 1 ELSE 0 END AS weather_condition_rainy,
    CASE WHEN weather_condition = 'Snowy' THEN 1 ELSE 0 END AS weather_condition_snowy,
    CASE WHEN weather_condition = 'Sunny' THEN 1 ELSE 0 END AS weather_condition_sunny,
    CASE WHEN weather_condition = 'Windy' THEN 1 ELSE 0 END AS weather_condition_windy,
    -- day_of_week
    CASE WHEN day_of_week = 'Sunday' THEN 1 ELSE 0 END AS day_of_week_sunday,
    CASE WHEN day_of_week = 'Monday' THEN 1 ELSE 0 END AS day_of_week_monday,
    CASE WHEN day_of_week = 'Tuesday' THEN 1 ELSE 0 END AS day_of_week_tuesday,
    CASE WHEN day_of_week = 'Wednesday' THEN 1 ELSE 0 END AS day_of_week_wednesday,
    CASE WHEN day_of_week = 'Thursday' THEN 1 ELSE 0 END AS day_of_week_thursday,
    CASE WHEN day_of_week = 'Friday' THEN 1 ELSE 0 END AS day_of_week_friday,
    CASE WHEN day_of_week = 'Saturday' THEN 1 ELSE 0 END AS day_of_week_saturday,
    -- traffic_level
    CASE WHEN traffic_level = 'High' THEN 1 ELSE 0 END AS traffic_level_high,
    CASE WHEN traffic_level = 'Medium' THEN 1 ELSE 0 END AS traffic_level_medium,
    CASE WHEN traffic_level = 'Low' THEN 1 ELSE 0 END AS traffic_level_low

    FROM
    `qwiklabs-gcp-01-a7e68aa7e5b0.emergency_calls.emergency_calls_table`
);

Query is running:   0%|          |

""


Create a model

In [26]:
%%bigquery create_model
CREATE OR REPLACE MODEL `qwiklabs-gcp-01-a7e68aa7e5b0.emergency_calls.emergency_calls_response_times_model`
OPTIONS (model_type='linear_reg',
input_label_cols=['response_time']) AS
SELECT * FROM `qwiklabs-gcp-01-a7e68aa7e5b0.emergency_calls.emergency_calls_table_encoded`
WHERE response_time IS NOT NULL AND call_timestamp BETWEEN '2023-01-01' AND '2023-11-30'

Query is running:   0%|          |

Evaluate the model

In [29]:
%%bigquery eval_model
SELECT
  *
FROM
  ML.EVALUATE(MODEL `qwiklabs-gcp-01-a7e68aa7e5b0.emergency_calls.emergency_calls_response_times_model`);

Query is running:   0%|          |

Downloading:   0%|          |

In [30]:
eval_model

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.745031,4.783751,0.014862,1.469866,0.82902,0.829173


Run the prediction for the rest of the year

In [31]:
%%bigquery prediction
SELECT * FROM ML.PREDICT (
    MODEL`qwiklabs-gcp-01-a7e68aa7e5b0.emergency_calls.emergency_calls_response_times_model`,
      (SELECT * FROM `qwiklabs-gcp-01-a7e68aa7e5b0.emergency_calls.emergency_calls_table_encoded`
      WHERE response_time IS NOT NULL AND call_timestamp BETWEEN '2023-12-01' AND '2023-12-31'))

Query is running:   0%|          |

Downloading:   0%|          |

In [32]:
prediction

,predicted_response_time,call_id,call_timestamp,location,time_of_day,distance_to_station,units_available,response_time,call_type_police,call_type_medical,...,day_of_week_sunday,day_of_week_monday,day_of_week_tuesday,day_of_week_wednesday,day_of_week_thursday,day_of_week_friday,day_of_week_saturday,traffic_level_high,traffic_level_medium,traffic_level_low
0,-5815.360641,29367,2023-12-01 00:02:58+00:00,Oakmont,0,7.82,9,14.15,0,0,...,0,0,0,0,0,1,0,0,0,1
1,-5805.119151,25304,2023-12-01 00:04:17+00:00,Downtown,0,23.42,9,19.89,0,0,...,0,0,0,0,0,1,0,0,0,1
2,-5813.143901,44026,2023-12-01 00:09:39+00:00,Downtown,0,9.34,3,15.28,0,0,...,0,0,0,0,0,1,0,0,0,1
3,-5804.784496,5824,2023-12-01 00:50:28+00:00,Downtown,0,25.21,12,23.66,0,0,...,0,0,0,0,0,1,0,0,0,1
4,-5806.804618,38908,2023-12-01 00:54:12+00:00,Highland,0,15.45,6,16.87,0,1,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4151,-220.499288,23938,2023-12-30 20:55:36+00:00,Brookfield,20,16.45,11,22.91,0,0,...,0,0,0,0,0,0,1,1,0,0
4152,-216.491128,18705,2023-12-30 21:12:04+00:00,Brookfield,21,24.47,10,24.52,1,0,...,0,0,0,0,0,0,1,1,0,0
4153,-221.390138,807,2023-12-30 22:07:17+00:00,Brookfield,22,21.18,1,19.92,1,0,...,0,0,0,0,0,0,1,1,0,0
4154,-219.887504,47290,2023-12-30 22:36:45+00:00,Maplewood,22,27.18,8,26.35,0,0,...,0,0,0,0,0,0,1,1,0,0


No bueno predictions!